In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Model
from keras.layers import Input, LSTM, Dense

Using TensorFlow backend.


In [5]:
batch_size = 64  # Batch size for training.
epochs = 50  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 1000  # Number of samples to train on.

In [6]:
#read from dataset and split into input and targets
df = pd.read_csv("cleaned_data.csv")
df.head()

,0,1,2
0,What kind of phones do you guys have,I have a pixel It is pretty great Much better ...,Does it really charge all the way in min
1,I have a pixel It is pretty great Much better ...,Does it really charge all the way in min,Pretty fast I have never timed it but it is un...
2,Does it really charge all the way in min,Pretty fast I have never timed it but it is un...,cool I have been thinking of getting one my ph...
3,What kind of phones do you guys have,Samsung Galaxy J It is my first cell phone and...,What do you think of it Anything you do not like
4,Samsung Galaxy J It is my first cell phone and...,What do you think of it Anything you do not like,I love it I cannot think of anything I do not ...


In [7]:
print(len(df))
list0 = df["0"].tolist()[:num_samples]
list1 = df["1"].tolist()[:num_samples]
list2 = df["2"].tolist()[:num_samples]
print(len(list0))
print(len(list1))
print(len(list2))

56297
1000
1000
1000


In [8]:
input_texts=[]
target_texts=[]
#match 0 and 1
for i in range(0,num_samples):
    input_texts.append(list0[i])
    target_texts.append(list1[i])

#match 1 and 2
for i in range(0,num_samples):
    input_texts.append(list1[i])
    target_texts.append(list2[i])

#match 0 and 2
for i in range(0,num_samples):
    input_texts.append(list0[i])
    target_texts.append(list2[i])
    

In [9]:
# len(input_texts)
# len(target_texts)
input_texts

['What kind of phones do you guys have',
 'I have a pixel It is pretty great Much better than what I had before',
 'Does it really charge all the way in   min',
 'What kind of phones do you guys have',
 'Samsung Galaxy J It is my first cell phone and I have had it for   months',
 'What do you think of it Anything you do not like',
 'What kind of phones do you guys have',
 'LG Optimus V I know it is old',
 'My friend told me to kill myself  ',
 'Do not kill yourself OP',
 'My friend told me to kill myself  ',
 'Fuck those losers I will we will be your friend',
 'I just won a state championship for debate',
 'No you did not',
 'Huh',
 'No you did not',
 'So I got a girlfriend the other day',
 'CONGRATS Hope you are as happy as you could possibly be  ',
 'So I got a girlfriend the other day',
 'That is so lovely Basically my gayass goals',
 'So I got a girlfriend the other day',
 'so mate you missed out the important bit did you stick it in herget stuck',
 'Um I am a girl Also Is that you

In [10]:
#a dictionary of vocab
vocab = set()
for sentence in (input_texts + target_texts):
#     print(sentence)
    words = sentence.split(" ")
    for word in words:
        if word not in vocab:
            vocab.add(word)

In [20]:
# #words in input
# input_char = set()
# for sentence in (input_texts):
#     words = sentence.split(" ")
#     for word in words:
#         if word not in input_char:
#             input_char.add(word)
            
            
# target_char = set()
# for sentence in (target_texts):
#     words = sentence.split(" ")
#     for word in words:
#         if word not in target_char:
#             target_char.add(word)

In [15]:
input_char = set()
target_char=set()

#grab sets of all the characters in inputs and targets
for line in input_texts:
    for char in line:
        if char not in input_char:
            input_char.add(char)
            
for line in target_texts:
    for char in line:
        if char not in target_char:
            target_char.add(char)

In [21]:
target_char

{'',
 'Peace',
 'ein',
 'bottom',
 'pickle',
 'looking',
 'feelin',
 'swirly',
 'Stealth',
 'awe',
 'spelling',
 'Indeed',
 'Take',
 'others',
 'elderly',
 'sammiches',
 'strong',
 'bonding',
 'lady',
 'thing',
 'make',
 'ceremony',
 'IMHO',
 'Oops',
 'reached',
 'enough',
 'faith',
 'entire',
 'play',
 'threads',
 'Being',
 'programs',
 'bully',
 'still',
 'haha',
 'loaf',
 'trail',
 'Heheheheheh',
 'ourselves',
 'tins',
 'Resident',
 'coffee',
 'choice',
 'summer',
 'Mmmm',
 'turnt',
 'hyperbole',
 'Ooh',
 'everytime',
 'cookie',
 'dammit',
 'along',
 'words',
 'after',
 'boob',
 'lose',
 'cream',
 'an',
 'she',
 'game',
 'fail',
 'picture',
 'MADE',
 'evil',
 'th',
 'absolutely',
 'bronchitis',
 'favourite',
 'stations',
 'ooh',
 'Already',
 'Choclate',
 'Slowly',
 'family',
 'sweet',
 'Hawaiian',
 'reply',
 'RAGE',
 'vs',
 'phase',
 'ahah',
 'Donnie',
 'prison',
 'leg',
 'Durin',
 'Wish',
 'shutin',
 'become',
 'chess',
 'air',
 'tv',
 'Doom',
 'Bradley',
 'Marry',
 'Noice',
 'swea

In [22]:
input_char = sorted(list(input_char))
target_char = sorted(list(target_char))

num_encoder_tokens = len(input_char)
num_decoder_tokens=len(target_char)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 3000
Number of unique input tokens: 2357
Number of unique output tokens: 2700
Max sequence length for inputs: 175
Max sequence length for outputs: 99


In [24]:
#enumerate the chars 
input_token_index = dict(
    [(char,i) for i,char in enumerate (input_char)]
)

target_token_index = dict(
    [(char,i) for i,char in enumerate (target_char)]
)

In [0]:
# input_token_index

In [25]:
#one hot vectorization of input and target sentences - 3D array
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros( #ahead by one time step?
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [26]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

KeyError: 'W'

In [60]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [61]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [62]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 50)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 50)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 314368      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  314368      input_2[0][0]                    
                                                                 lstm_1[0][1]               

In [65]:
history=model.fit(
        [encoder_input_data, decoder_input_data], 
        decoder_target_data,
        batch_size=batch_size,
        epochs=epochs,
        verbose=0,
        validation_split=0.2)

KeyboardInterrupt: 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [0]:
# Save model
model.save('s2s.h5')

In [0]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [0]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    # target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [0]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: im pleased you two are so impressed i find the whole
Decoded sentence: h                                                                                                                                                                                                                                                                                                                                                                                                                                
-
Input sentence: he is let us go
Decoded sentence: h                                                                                                                                                                                                                                                                                                                                                                                                                                
-
Input sentence: witho